## ML Project Part 2

### Write a function that estimates the transition parameters from the training set using MLE

In [1]:
"""
we need 2 things:
every tag pair for consecutive words
every tag
"""

from collections import defaultdict

# helper to read data by sentences
def read_sentences(file_path):
    sentences = []
    sentence = []

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line:
                word, tag = line.split()
                sentence.append(tag)
            else:
                if sentence:
                    sentences.append(sentence)
                    sentence = []

    if sentence:
        sentences.append(sentence)

    return sentences

# function to estimate transition parameters
def estimate_transition_parameters(sentences):
    transition_counts = defaultdict(lambda: defaultdict(int))
    tag_counts = defaultdict(int)

    for sentence in sentences:
        tags = ['START'] + sentence + ['STOP']
        for i in range(1, len(tags)):
            prev_tag = tags[i - 1]
            curr_tag = tags[i]
            transition_counts[prev_tag][curr_tag] += 1
            tag_counts[prev_tag] += 1  # count only as previous tag

    transition_probs = {}

    for prev_tag in transition_counts:
        transition_probs[prev_tag] = {}
        for curr_tag in transition_counts[prev_tag]:
            transition_probs[prev_tag][curr_tag] = transition_counts[prev_tag][curr_tag] / tag_counts[prev_tag]

    return transition_probs

if __name__ == "__main__":

    sentences = read_sentences("EN/train")
    transition_probs = estimate_transition_parameters(sentences)

    # eg q(B-NP | START)
    print(transition_probs["START"].get("B-NP", 0))

    # eg q(STOP | I-NP)
    print(transition_probs["I-NP"].get("STOP", 0))


0.6480490669450607
0.000787675624187137


### Implementing the Viterbi algorithm

THIS IS WRONG AND INCOMPLETE

In [7]:
"""
given a sequence of words x1,...,xn, find the most likely tag sequence y1*,....,yn* using the Viterbi algo
we need to maintain 2 things:
pi[i][tag]: max log-prob of best path to position i ending in tag
bp[i][tag]: best previous tag to reach tag at position i
"""

import math

def viterbi_decode(dev_path, output_path, tag_list, emission_probs, transition_probs, known_words):
    """
    function applies viterbi algo to each sentence in the dev set and outputs best tag sequence
    """
    # first, lets collect lines from dev.in and group them into sentences
    with open(dev_path, 'r') as f:
        lines = f.readlines()

    sentences = []
    sentence = []
    for line in lines:
        word = line.strip()
        if word == '':
            if sentence:
                sentences.append(sentence)
                sentence = []
        else:
            sentence.append(word)
    if sentence:
        sentences.append(sentence)

    with open(output_path, 'w') as f_out:
        for sentence in sentences:
            n = len(sentence)
            pi = [{} for _ in range(n + 1)] # pi[i][tag] stores the best log-prob for reaching tag at position i
            bp = [{} for _ in range(n + 1)] # bp[i][tag] stores the backpointer - best previous tag

            pi[0]["START"] = 0.0

            for i in range(1, n + 1):
                word = sentence[i - 1]
                word_key = word if word in known_words else "#UNK#" # if word not in training set, replace with #UNK#

                for curr_tag in tag_list:
                    if word_key not in emission_probs[curr_tag]:
                        continue

                    best_score = float("-inf")
                    best_prev_tag = None

                    for prev_tag in pi[i - 1]:
                        if curr_tag not in transition_probs.get(prev_tag, {}):
                            continue
                        
                        # calc score for log-prob for best path to curr_tag at position i
                        score = (
                            pi[i - 1][prev_tag]
                            + math.log(transition_probs[prev_tag][curr_tag])
                            + math.log(emission_probs[curr_tag][word_key])
                        )

                        if score > best_score:
                            best_score = score
                            best_prev_tag = prev_tag

                    # update dynamic programming table & backpointer
                    if best_prev_tag is not None:
                        pi[i][curr_tag] = best_score
                        bp[i][curr_tag] = best_prev_tag

            # termination: pick best final tag
            best_final_score = float("-inf")
            best_final_tag = None

            for tag in pi[n]:
                if "STOP" in transition_probs.get(tag, {}):
                    score = pi[n][tag] + math.log(transition_probs[tag]["STOP"])
                    if score > best_final_score:
                        best_final_score = score
                        best_final_tag = tag

            # Fallback if no STOP transition found
            if best_final_tag is None:
                best_final_tag = max(pi[n], key=pi[n].get)  # use tag with highest score at final position

            # backtracking: get full tag sequence
            tags = [best_final_tag]
            for i in range(n, 1, -1):
                tags.insert(0, bp[i][tags[0]])

            for word, tag in zip(sentence, tags):
                f_out.write(f"{word} {tag}\n")
            f_out.write("\n")

In [8]:
## THIS IS THE FUNCTIONS FROM PART 1

# function to read data
def read_training_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip(): # not an empty line
                word, tag = line.strip().split()
                data.append((word, tag))
    return data

# function to estimate emission parameters
def estimate_emission_parameters(data):
    count_y = defaultdict(int) # count how many times y appears overall
    count_y_to_x = defaultdict(lambda: defaultdict(int))  # count how many times word x is tagged with y
    
    for word, tag in data:
        count_y[tag] += 1
        count_y_to_x[tag][word] += 1

    emission_probs = {}  # e(x|y)

    for tag in count_y_to_x:
        emission_probs[tag] = {}
        for word in count_y_to_x[tag]:
            emission_probs[tag][word] = count_y_to_x[tag][word] / count_y[tag]
    
    return emission_probs

def replace_rare_words(data, word_counts, k=3):
    new_data = []
    for word, tag in data:
        if word_counts[word] < k:
            new_data.append(('#UNK#', tag))
        else:
            new_data.append((word, tag))
    return new_data

# function to count word frequencies
def count_word_frequencies(data):
    word_counts = defaultdict(int)
    for word, _ in data:
        word_counts[word] += 1
    return word_counts

In [9]:
# example usage
if __name__ == "__main__":
    train_path = "EN/train"
    dev_path = "EN/dev.in"
    output_path = "EN/dev.p2.out"

    # Prepare emission
    data = read_training_data(train_path)
    word_counts = count_word_frequencies(data)
    smoothed_data = replace_rare_words(data, word_counts, k=3)
    emission_probs = estimate_emission_parameters(smoothed_data)
    known_words = set(word for word, _ in smoothed_data)

    # Prepare transitions
    sentences = read_sentences(train_path)
    transition_probs = estimate_transition_parameters(sentences)

    # Get list of possible tags
    tag_list = list(emission_probs.keys())

    # Run Viterbi
    viterbi_decode(dev_path, output_path, tag_list, emission_probs, transition_probs, known_words)


ValueError: max() arg is an empty sequence